In [1]:
from pathlib import Path
from zmax_real_time.datasets.donders_2022 import Donders2022
from zmax_real_time.utils import load_yaml_config

CONFIG_PATH =  Path("../configs/donders_2022.yaml")
config = load_yaml_config(CONFIG_PATH)
print(config)
dataset = Donders2022(**config["datasets"]["donders_2022"])
print(len(dataset))
print(dataset.recordings.head)

{'datasets': {'donders_2022': {'data_dir': '/project/3013097.06/Data', 'channel_names': ['EEG L', 'EEG R'], 'annotations': ['sleep_stage']}}}
76
<bound method NDFrame.head of     subject_id  session_id                                        annotations  \
0            3           1  /project/3013097.06/Data/s3/n1/zmax/s3 n1_psg.txt   
1            3           8  /project/3013097.06/Data/s3/n8/zmax/s3 n8_psg.txt   
2            1           1  /project/3013097.06/Data/s1/n1/zmax/s1 n1_psg.txt   
3            1           8  /project/3013097.06/Data/s1/n8/zmax/s1 n8_psg.txt   
4            1          15  /project/3013097.06/Data/s1/n15/zmax/s1 n15_ps...   
..         ...         ...                                                ...   
71          36           1  /project/3013097.06/Data/s36/n1/zmax/s36 n1_ps...   
72          36           8  /project/3013097.06/Data/s36/n8/zmax/s36 n8_ps...   
73          37          15  /project/3013097.06/Data/s37/n15/zmax/s37 n15_...   
74          37 

In [2]:
x = dataset.recordings.iloc[0]
x["EEG R"]

PosixPath('/project/3013097.06/Data/s3/n1/zmax/EEG R.edf')

In [ ]:
import mne

raw = mne.io.read_raw_edf(x["EEG R"])

In [1]:
import pandas as pd

DATA_FILE = "../data/yasa/zmax.parquet"

df = pd.read_parquet(DATA_FILE)

In [2]:
unique_counts = {}
for dataset in df.index.get_level_values(0).unique():
    subset = df.loc[dataset]
    unique_combinations = subset.index.unique()
    unique_counts[dataset] = len(unique_combinations)

for dataset, count in unique_counts.items():
    print(f"{dataset}: {count}")

total_count = sum(unique_counts.values())
print(f"Total unique combinations across all datasets: {total_count}")

Donders2018: 6
Donders2022: 72
Karolinska: 13
QS: 43
Total unique combinations across all datasets: 134


In [2]:
cols_all = df.columns
cols_time = cols_all[cols_all.str.startswith('time_')].tolist()
# EEG also includes the time columns
cols_eeg = cols_all[cols_all.str.startswith('eeg_')].tolist() + cols_time  
cols_eog = cols_all[cols_all.str.startswith('eog_')].tolist()

In [3]:
from lightgbm import LGBMClassifier

params = dict(
    boosting_type='gbdt',
    n_estimators=400,
    max_depth=5,
    num_leaves=90,
    colsample_bytree=0.5,
    importance_type='gain',
)
    

ImportError: libscipy_openblas-c128ec02.so: failed to map segment from shared object

In [ ]:
class_weight = "custom"

if class_weight == "custom":
    # See output/classifiers/gridsearch_class_weights.xlsx
    params['class_weight'] = {'N1': 2.2, 'N2': 1, 'N3': 1.2, 'R': 1.4, 'W': 1}
else:
    params['class_weight'] = class_weight